# Testing the API

Satellite data starts being recored on {'date': '2015-06-13'}

In [1]:
import requests
import os
from IPython.display import Image

In [2]:
# All Data
url = 'https://epic.gsfc.nasa.gov/api/natural/all'

# Most Recent
url = 'https://epic.gsfc.nasa.gov/api/natural/'

# URL by date
date = '20180428'
url = f'https://epic.gsfc.nasa.gov/api/enhanced/date/{date}'
res = requests.get(url)

In [3]:
# Example data retrieved
print(f'There are {len(res.json())} images available')
res.json()[0]

There are 22 images available


{'identifier': '20180428005516',
 'caption': "This image was taken by NASA's EPIC camera onboard the NOAA DSCOVR spacecraft",
 'image': 'epic_RGB_20180428005516',
 'version': '02',
 'centroid_coordinates': {'lat': 21.489258, 'lon': 165.871582},
 'dscovr_j2000_position': {'x': 1082471.700001,
  'y': 765002.975361,
  'z': 519622.982812},
 'lunar_j2000_position': {'x': -371472.817493,
  'y': -85970.908502,
  'z': 29631.801617},
 'sun_j2000_position': {'x': 119549193.299457,
  'y': 84022262.300594,
  'z': 36423693.849281},
 'attitude_quaternions': {'q0': 0.1799,
  'q1': 0.04705,
  'q2': -0.936977,
  'q3': 0.295793},
 'date': '2018-04-28 00:50:27',
 'coords': {'centroid_coordinates': {'lat': 21.489258, 'lon': 165.871582},
  'dscovr_j2000_position': {'x': 1082471.700001,
   'y': 765002.975361,
   'z': 519622.982812},
  'lunar_j2000_position': {'x': -371472.817493,
   'y': -85970.908502,
   'z': 29631.801617},
  'sun_j2000_position': {'x': 119549193.299457,
   'y': 84022262.300594,
   'z': 36

In [4]:
# Available data time and coords
fetch = 3
points = []
for e in res.json()[:fetch]:
    points.append((e['image'], e['centroid_coordinates'], e['date']))
points

[('epic_RGB_20180428005516',
  {'lat': 21.489258, 'lon': 165.871582},
  '2018-04-28 00:50:27'),
 ('epic_RGB_20180428020044',
  {'lat': 21.489258, 'lon': 149.458008},
  '2018-04-28 01:55:55'),
 ('epic_RGB_20180428030612',
  {'lat': 21.533203, 'lon': 133.132324},
  '2018-04-28 03:01:23')]

In [5]:
# Access the image
size = {'big':'.png', 'med':'jpg'}
collection = {'nat': 'natural', 'enh':'enhanced'}
img = f"https://epic.gsfc.nasa.gov/archive/{collection['enh']}/{date[:4]}/{date[4:6]}/{date[6:8]}/{size['med']}/{res.json()[0]['image']}.{size['med']}"
print(img)
#display(Image(url=img))

https://epic.gsfc.nasa.gov/archive/enhanced/2018/04/28/jpg/epic_RGB_20180428005516.jpg


In [6]:
# Access the image
size = {'big':'.png', 'med':'jpg'}
collection = {'nat': 'natural', 'enh':'enhanced'}
for e in res.json():
    img = f"https://epic.gsfc.nasa.gov/archive/{collection['enh']}/{date[:4]}/{date[4:6]}/{date[6:8]}/png/{e['image']}.{size['big']}"
    #display(Image(url=img))

In [7]:
# Example data retrieved
print(res.json()[0]['identifier'])
YYYYMMDD = res.json()[0]['identifier']
year = YYYYMMDD[:4]
month = YYYYMMDD[4:6]
day = YYYYMMDD[6:8]
print(year, month, day)

20180428005516
2018 04 28


# Basic getSat() by date

In [8]:
## VERSION 1
def getSat(img_type, YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    # Define types of collections
    collection = {'nat': 'natural', 'enh':'enhanced'}
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collection[img_type]}/date/{YYYY}-{MM}-{DD}"
    res = requests.get(metadata, params=queryParams)
    print(res.status_code, res.url)

    # Construct the resource url
    img = f"{url}/archive/{collection[img_type]}/{YYYY}/{MM}/{DD}/png/{e['image']}.{size['big']}"

    # Return json from requests' response body
    return res.json()

#getSat('nat', '2020', '04', '09')[1]

In [9]:
## VERSION 02
# Define when
def getSat(img_type, YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    # Define types of collections
    collection = {'nat': 'natural', 'enh':'enhanced'}
    date = f"{YYYY}-{MM}-{DD}"
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collection[img_type]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    # Construct the resource url
    #img = f"{url}/archive/{collection[img_type]}/{YYYY}/{MM}/{DD}/png/{e['image']}.{size['big']}"
    
    #Return relevant data as list
    return data

#getSat('nat', '2020', '03', '10')

In [10]:
## VERSION 03
# Define when
def getSat(collection, img_type,YYYY, MM, DD, queryParams=dict(), apiKey=""):
    
    date = f"{YYYY}-{MM}-{DD}"
    
    # Define types of image collections and size 
    collections = {'nat': 'natural', 'enh':'enhanced'}

    size = {'png': ['png', 'png'],
            'jpg': ['jpg', 'jpg'],
            'thumb': ['thumbs', 'jpg']}
    
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collections[collection]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    
    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    
    # Construct the resource url
    if len(data) > 0:
        for e in data:
            print(e)
            img = f"{host}/archive/{collections[collection]}/{YYYY}/{MM}/{DD}/{size[img_type][0]}/{e[0]}.{size[img_type][1]}"
            print(img)
    #Return relevant data as list
    return data

#getSat('nat', 'thumb', '2020', '03', '10')

In [11]:
## VERSION 04
def getSat(collection, img_type, YYYY,MM,DD, queryParams=dict()):
    
    date = f"{YYYY}-{MM}-{DD}"
    
    # Define types of image collections and size 
    collections = {'nat': 'natural', 'enh':'enhanced'}

    size = {'png': ['png', 'png'],
            'jpg': ['jpg', 'jpg'],
            'thumb': ['thumbs', 'jpg']}
    
    
    # Request API metadata using query parameters
    host = f'https://epic.gsfc.nasa.gov'
    metadata = f"{host}/api/{collections[collection]}/date/{date}"
    res = requests.get(metadata, params=queryParams)
    
    
    # Store reelevant data from requests' response body
    data = []
    for e in res.json():
        data.append((e['image'], e['centroid_coordinates'], e['date']))
    
    
    # Report status of query response
    print(res.status_code, res.url)
    print(f"There are {len(res.json())} satellite images available for this date: {date}")

    
    # Construct the image resource url
    satellite_images = []
    if len(data) > 0:
        for e in data:
            img = f"{host}/archive/{collections[collection]}/{YYYY}/{MM}/{DD}/{size[img_type][0]}/{e[0]}.{size[img_type][1]}"
            satellite_images.append([img, e[1]])
    #Return relevant data as list
    return satellite_images

images = getSat('enh', 'thumb', '2020', '03', '01')

200 https://epic.gsfc.nasa.gov/api/enhanced/date/2020-03-01
There are 12 satellite images available for this date: 2020-03-01


In [12]:
display(Image(url=images[0][0]))

# Working with cleaned dataset dates

In [13]:
import pandas as pd
import numpy as np
import datetime

In [14]:
df = pd.read_csv('OUTPUT/volcanic-explosions.csv')
df = df.drop(columns='Unnamed: 0')

In [15]:
# Select an event on the dataframe for which we know there is available data in the API
event_date = df.iloc[20]['start']
event_date = event_date.split('-')
print(event_date)

# GET the API image and coordinates
available_data = getSat('nat', 'png', *event_date)
print(available_data[0])
print(available_data[-1])

['2019', '06', '22']
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-06-22
There are 22 satellite images available for this date: 2019-06-22
['https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/png/epic_1b_20190622004555.png', {'lat': 20.357666, 'lon': 173.430176}]
['https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/png/epic_1b_20190622234040.png', {'lat': 20.126953, 'lon': -170.551758}]


# Finding the satellite images for the START of each eruptive phase

In [16]:
# Access the START dates on each eruptive phase
for phase in df['start'][5:20]:
    print(phase)
    print(getSat('nat', 'thumb', *phase.split('-')))

2019-12-09
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-12-09
There are 0 satellite images available for this date: 2019-12-09
[]
2019-12-07
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-12-07
There are 0 satellite images available for this date: 2019-12-07
[]
2019-12-05
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-12-05
There are 0 satellite images available for this date: 2019-12-05
[]
2019-11-07
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-11-07
There are 0 satellite images available for this date: 2019-11-07
[]
2019-11-02
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-11-02
There are 0 satellite images available for this date: 2019-11-02
[]
2019-10-24
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-10-24
There are 0 satellite images available for this date: 2019-10-24
[]
2019-10-13
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-10-13
There are 0 satellite images available for this date: 2019-10-13
[]
2019-08-07
200 https://epic.gsfc.n

In [17]:
# Access the START dates on each eruptive phase
for phase in df['start'][::15]:
    print()
    print(phase)
    phase_data = getSat('nat', 'thumb', *phase.split('-'))
    if len(phase_data) > 0:
        print('This is the first available image for that date')
        print(phase_data[0])


2020-02-10
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-02-10
There are 0 satellite images available for this date: 2020-02-10

2019-07-23
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-07-23
There are 0 satellite images available for this date: 2019-07-23

2019-02-18
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-02-18
There are 13 satellite images available for this date: 2019-02-18
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2019/02/18/thumbs/epic_1b_20190218001751.jpg', {'lat': -19.533691, 'lon': 171.254883}]

2018-09-08
200 https://epic.gsfc.nasa.gov/api/natural/date/2018-09-08
There are 13 satellite images available for this date: 2018-09-08
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2018/09/08/thumbs/epic_1b_20180908005515.jpg', {'lat': 7.954102, 'lon': 155.874023}]

2018-04-27
200 https://epic.gsfc.nasa.gov/api/natural/date/2018-04-27
There are 17 satellite im

# Reach API to get the links for each volcanic eruption in the dataset

In [18]:
# Access the START dates on each eruptive phase
img_urls = []
sat_lats = []
sat_lons = []

for phase_start in df['start']:
    print()
    print(phase_start)
    phase_data = getSat('nat', 'thumb', *phase_start.split('-'))
    if len(phase_data) > 0:
        print('This is the first available image for that date')
        print(phase_data[0])
        img_urls.append(phase_data[0][0])
        sat_lats.append(phase_data[0][1]['lat'])
        sat_lons.append(phase_data[0][1]['lon'])

    else:
        img_urls.append('no-img')
        sat_lats.append('0')
        sat_lons.append('0')

        #img_urls.append(['no-img', {'lat':0, 'lon':0}])

df['start_img'] = img_urls


2020-02-10
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-02-10
There are 0 satellite images available for this date: 2020-02-10

2020-01-31
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-01-31
There are 0 satellite images available for this date: 2020-01-31

2020-01-12
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-01-12
There are 0 satellite images available for this date: 2020-01-12

2020-01-12
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-01-12
There are 0 satellite images available for this date: 2020-01-12

2020-01-11
200 https://epic.gsfc.nasa.gov/api/natural/date/2020-01-11
There are 0 satellite images available for this date: 2020-01-11

2019-12-09
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-12-09
There are 0 satellite images available for this date: 2019-12-09

2019-12-07
200 https://epic.gsfc.nasa.gov/api/natural/date/2019-12-07
There are 0 satellite images available for this date: 2019-12-07

2019-12-05
200 https://epic.gsfc.nasa.gov/api/n

200 https://epic.gsfc.nasa.gov/api/natural/date/2018-10-26
There are 13 satellite images available for this date: 2018-10-26
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2018/10/26/thumbs/epic_1b_20181026010436.jpg', {'lat': -6.943359, 'lon': 165.651855}]

2018-10-21
200 https://epic.gsfc.nasa.gov/api/natural/date/2018-10-21
There are 2 satellite images available for this date: 2018-10-21
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2018/10/21/thumbs/epic_1b_20181021041239.jpg', {'lat': -4.943848, 'lon': 117.246094}]

2018-10-04
200 https://epic.gsfc.nasa.gov/api/natural/date/2018-10-04
There are 13 satellite images available for this date: 2018-10-04
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2018/10/04/thumbs/epic_1b_20181004004554.jpg', {'lat': 1.713867, 'lon': 163.015137}]

2018-10-02
200 https://epic.gsfc.nasa.gov/api/natural/date/2018-1

200 https://epic.gsfc.nasa.gov/api/natural/date/2018-04-03
There are 10 satellite images available for this date: 2018-04-03
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2018/04/03/thumbs/epic_1b_20180403000830.jpg', {'lat': 6.628418, 'lon': 170.939941}]

2018-04-01
200 https://epic.gsfc.nasa.gov/api/natural/date/2018-04-01
There are 13 satellite images available for this date: 2018-04-01
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2018/04/01/thumbs/epic_1b_20180401003633.jpg', {'lat': 5.273438, 'lon': 163.806152}]

2018-03-01
200 https://epic.gsfc.nasa.gov/api/natural/date/2018-03-01
There are 12 satellite images available for this date: 2018-03-01
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2018/03/01/thumbs/epic_1b_20180301004554.jpg', {'lat': -14.143066, 'lon': 164.128418}]

2018-02-25
200 https://epic.gsfc.nasa.gov/api/natural/date/2018-

200 https://epic.gsfc.nasa.gov/api/natural/date/2017-05-23
There are 22 satellite images available for this date: 2017-05-23
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2017/05/23/thumbs/epic_1b_20170523000830.jpg', {'lat': 26.242676, 'lon': -177.502441}]

2017-05-10
200 https://epic.gsfc.nasa.gov/api/natural/date/2017-05-10
There are 22 satellite images available for this date: 2017-05-10
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2017/05/10/thumbs/epic_1b_20170510002713.jpg', {'lat': 24.32373, 'lon': 172.001953}]

2017-04-30
200 https://epic.gsfc.nasa.gov/api/natural/date/2017-04-30
There are 21 satellite images available for this date: 2017-04-30
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2017/04/30/thumbs/epic_1b_20170430003634.jpg', {'lat': 20.588379, 'lon': 165.717773}]

2017-04-18
200 https://epic.gsfc.nasa.gov/api/natural/date/2017

200 https://epic.gsfc.nasa.gov/api/natural/date/2016-09-13
There are 13 satellite images available for this date: 2016-09-13
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2016/09/13/thumbs/epic_1b_20160913000830.jpg', {'lat': 4.87793, 'lon': 169.40918}]

2016-09-13
200 https://epic.gsfc.nasa.gov/api/natural/date/2016-09-13
There are 13 satellite images available for this date: 2016-09-13
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2016/09/13/thumbs/epic_1b_20160913000830.jpg', {'lat': 4.87793, 'lon': 169.40918}]

2016-09-11
200 https://epic.gsfc.nasa.gov/api/natural/date/2016-09-11
There are 12 satellite images available for this date: 2016-09-11
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2016/09/11/thumbs/epic_1b_20160911003633.jpg', {'lat': 5.053711, 'lon': 162.883301}]

2016-08-03
200 https://epic.gsfc.nasa.gov/api/natural/date/2016-08-03


200 https://epic.gsfc.nasa.gov/api/natural/date/2015-11-14
There are 12 satellite images available for this date: 2015-11-14
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2015/11/14/thumbs/epic_1b_20151114003634.jpg', {'lat': -10.012207, 'lon': 163.161621}]

2015-11-12
200 https://epic.gsfc.nasa.gov/api/natural/date/2015-11-12
There are 13 satellite images available for this date: 2015-11-12
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2015/11/12/thumbs/epic_1b_20151112010437.jpg', {'lat': -9.228516, 'lon': 155.544434}]

2015-10-30
200 https://epic.gsfc.nasa.gov/api/natural/date/2015-10-30
There are 11 satellite images available for this date: 2015-10-30
This is the first available image for that date
['https://epic.gsfc.nasa.gov/archive/natural/2015/10/30/thumbs/epic_1b_20151030005515.jpg', {'lat': -4.87793, 'lon': 154.973145}]

2015-10-25
200 https://epic.gsfc.nasa.gov/api/natural/date/2015

In [19]:
img_urls

['no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'no-img',
 'https://epic.gsfc.nasa.gov/archive/natural/2019/06/26/thumbs/epic_1b_20190626001751.jpg',
 'https://epic.gsfc.nasa.gov/archive/natural/2019/06/24/thumbs/epic_1b_20190624041139.jpg',
 'https://epic.gsfc.nasa.gov/archive/natural/2019/06/22/thumbs/epic_1b_20190622004555.jpg',
 'https://epic.gsfc.nasa.gov/archive/natural/2019/06/11/thumbs/epic_1b_20190611003634.jpg',
 'https://epic.gsfc.nasa.gov/archive/natural/2019/06/01/thumbs/epic_1b_20190601004554.jpg',
 'https://epic.gsfc.nasa.gov/archive/natural/2019/05/16/thumbs/epic_1b_20190516002712.jpg',
 'https://epic.gsfc.nasa.gov/archive/natural/2019/05/11/thumbs/epic_1b_20190511010437.jpg',
 'https://epic.gsfc.nasa.gov/archive/natural/2019/04/16/thumbs/epic_1b_20190416000830.jpg',
 'https://epic.gsfc.nasa.gov/archive/natural/2019/04/09/thumbs/epi

In [26]:
df['sat_lats'] = sat_lats
df['sat_lons'] = sat_lons

In [30]:
df['sat_lons'].value_counts()

0             49
-179.94873     3
155.302734     2
155.874023     2
171.97998      2
              ..
173.671875     1
175.041504     1
176.000977     1
177.663574     1
167.299805     1
Name: sat_lons, Length: 126, dtype: int64

In [20]:
0/0

ZeroDivisionError: division by zero

In [27]:
df.to_csv('OUTPUT/enriched-data.csv')

In [29]:
df[20:50]

,v_num,v_name,erup_num,erup_cat,vei,start_y,start_m,start_d,end_y,end_m,end_d,lat,lon,start,end,delta,start_img,sat_lats,sat_lons
20,290250,Raikoke,22321,Confirmed Eruption,NaN,2019,6,22,2019,7,1,48.292,153.250,2019-06-22,2019-07-01,9 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,20.3577,173.43
21,233020,"Fournaise, Piton de la",22318,Confirmed Eruption,NaN,2019,6,11,2019,10,27,-21.244,55.708,2019-06-11,2019-10-27,138 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,22.6318,178.857
22,311120,Great Sitkin,22322,Confirmed Eruption,NaN,2019,6,1,2019,6,7,52.076,-176.130,2019-06-01,2019-06-07,6 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,24.2944,177.664
23,290240,Sarychev Peak,22320,Confirmed Eruption,2.0,2019,5,16,2019,10,7,48.092,153.200,2019-05-16,2019-10-07,144 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,25.5542,-177.891
24,341040,Colima,22323,Confirmed Eruption,NaN,2019,5,11,2019,7,12,19.514,-103.620,2019-05-11,2019-07-12,62 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,25.2686,171.98
25,282110,Asosan,22317,Confirmed Eruption,NaN,2019,4,16,2020,2,17,32.884,131.104,2019-04-16,2020-02-17,307 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,17.3584,178.857
26,300260,Klyuchevskoy,22310,Confirmed Eruption,NaN,2019,4,9,2019,7,2,56.056,160.642,2019-04-09,2019-07-02,84 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,13.1396,176.968
27,352090,Sangay,22312,Confirmed Eruption,NaN,2019,3,26,2020,2,19,-2.005,-78.341,2019-03-26,2020-02-19,330 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,3.20801,173.738
28,344020,San Cristobal,22311,Confirmed Eruption,NaN,2019,3,4,2019,3,4,12.702,-87.004,2019-03-04,2019-03-04,0 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,-12.085,170.178
29,233020,"Fournaise, Piton de la",22307,Confirmed Eruption,NaN,2019,2,18,2019,3,10,-21.244,55.708,2019-02-18,2019-03-10,20 days 00:00:00.000000000,https://epic.gsfc.nasa.gov/archive/natural/201...,-19.5337,171.255


In [ ]:
df = pd.DataFrame(date_rng, columns=['date'])
df['data'] = np.random.randint(0,100, size=(len(date_rng)))
df.head()

In [ ]:
date_rng = pd.date_range(start='1/1/2015', end='10/04/2020', freq='D')
date_rng

In [ ]:
df['datetime'] = pd.to_datetime(df.date)
df = df.set_index('datetime')

#list only the second day of each month
#df[df.index.day == 2]

# calculate mean of each year
df.resample('M').mean()